In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('../input/crop-recommendation-dataset/Crop_recommendation.csv')

In [3]:
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [5]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [6]:
dummy_y

array([[0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2, random_state=8)

In [9]:
X_train.shape, y_train.shape,X_test.shape, y_test.shape

((1760, 7), (1760, 22), (440, 7), (440, 22))

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Now Building the Artificial Neurak Network

In [11]:
ann = tf.keras.models.Sequential()

In [12]:
ann.add(tf.keras.layers.Dense(7))

In [13]:
ann.add(tf.keras.layers.Dense(64, activation='relu'))

In [14]:
ann.add(tf.keras.layers.Dense(32, activation='relu'))

In [15]:
ann.add(tf.keras.layers.Dense(units=22, activation='softmax'))

# Compiling the ANN

In [16]:
ann.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

# Training the ANN on training Set

In [17]:
ann.fit(X_train, y_train, batch_size=64, epochs=500)

Epoch 1/500
28/28 [==============================] - 1s 1ms/step - loss: 2.9968 - accuracy: 0.1013
Epoch 2/500
28/28 [==============================] - 0s 1ms/step - loss: 2.6958 - accuracy: 0.2126
Epoch 3/500
28/28 [==============================] - 0s 1ms/step - loss: 2.2881 - accuracy: 0.3754
Epoch 4/500
28/28 [==============================] - 0s 1ms/step - loss: 1.8101 - accuracy: 0.5304
Epoch 5/500
28/28 [==============================] - 0s 1ms/step - loss: 1.2974 - accuracy: 0.6803
Epoch 6/500
28/28 [==============================] - 0s 1ms/step - loss: 0.9241 - accuracy: 0.7833
Epoch 7/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6874 - accuracy: 0.8506
Epoch 8/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5415 - accuracy: 0.8843
Epoch 9/500
28/28 [==============================] - 0s 1ms/step - loss: 0.4487 - accuracy: 0.9032
Epoch 10/500
28/28 [==============================] - 0s 1ms/step - loss: 0.3650 - accuracy: 0.9274
Epoch 11/

In [18]:
results = ann.evaluate(X_test, y_test)

14/14 [==============================] - 0s 1ms/step - loss: 0.0804 - accuracy: 0.9727


In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = accuracy_score(y_test, y_pred)
print(cm)

0.9727272727272728
